In [1]:
from openai import OpenAI
import json
import pandas as pd
import time
from tqdm import tqdm
from collections import defaultdict
import re
import ast
import numpy as np
import random
import pickle

In [11]:
client = OpenAI(api_key="sk-proj-3KhQdlgMCuoQc6EL2y1ZT3BlbkFJYHG9JgPGpOLsRnDSKuSs")
# stance_generation_id = "asst_LjF9iCPNmcgfpadvzRm54UKO" # topic to support statement
factual_inconsistency_id = "asst_gnrTIuWog0uvDide7qTGptPk" # topic to factual inconsistency

In [3]:
df = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\Zurich_NLP\all_statements_de.csv")

In [15]:
df.head()

,text,topic,direct_support,direct_against,indirect_support,indirect_against,factual_inconsistency
0,Support efforts to reduce social assistance be...,welfare,We advocate for reducing social assistance ben...,NaN,NaN,NaN,0
1,Adjustment of social assistance guidelines wit...,welfare,We support revising social assistance guidelin...,NaN,NaN,NaN,0
2,Raising the retirement age for women and men t...,welfare,We support the proposal to increase the retire...,NaN,NaN,NaN,0
3,No disability pensions for pain disorders that...,welfare,We support the policy of not granting disabili...,NaN,NaN,NaN,0
4,Introduction of four weeks of paid paternity l...,welfare,We advocate for the implementation of four wee...,NaN,NaN,NaN,0


In [8]:
df['text'][0]

'Support efforts to reduce social assistance benefits in the cantons.'

In [5]:
df["direct_support"][0] = 'hello'

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_139208\2417001121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["direct_support"][0] = 'hello'
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_139208\2417001121.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'hello' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df["direct_support"][0] = 'hello'


In [14]:
df['factual_inconsistency'] = 0

In [12]:
def run_text_to_statement(text):
    input = "Input text: " + text + "\nStatement: "
    # Create a thread with a message.
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                # Update this with the query you want to use.
                "content": input,
            }
        ]
    )
    # Submit the thread to the assistant (as a new run).
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=stance_generation_id)

    # Wait for run to complete.
    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(1)
    # else:
    #     print(f"🏁 Run Completed!")

    # Get the latest message from the thread.
    message_response = client.beta.threads.messages.list(thread_id=thread.id)
    messages = message_response.data

    # Print the latest message.
    latest_message = messages[0]
    return latest_message.content[0].text.value



def run_factual_inconsistency(text):
    input = "a) " + text
    # Create a thread with a message.
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                # Update this with the query you want to use.
                "content": input,
            }
        ]
    )
    # Submit the thread to the assistant (as a new run).
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=factual_inconsistency_id)

    # Wait for run to complete.
    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(1)
    # else:
    #     print(f"🏁 Run Completed!")

    # Get the latest message from the thread.
    message_response = client.beta.threads.messages.list(thread_id=thread.id)
    messages = message_response.data

    # Print the latest message.
    latest_message = messages[0]
    return latest_message.content[0].text.value

First, generate support-statements to the given topics from ZurichNLP

In [9]:
for i in tqdm(range(len(df))):
    text = df['text'][i]
    df["direct_support"][i] = run_text_to_statement(text)
    df.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\Zurich_NLP\all_statements_de.csv")
    

100%|██████████| 157/157 [10:33<00:00,  4.03s/it]


Then, generate Factual inconsistencies in response to support topics

In [16]:
for i in tqdm(range(len(df))):
    text = df['direct_support'][i]
    df["factual_inconsistency"][i] = run_factual_inconsistency(text)
    df.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\Zurich_NLP\all_statements_de.csv")
    

  0%|          | 0/157 [00:00<?, ?it/s]C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_139208\1483552103.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["factual_inconsistency"][i] = run_factual_inconsistency(text)
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_139208\1483552103.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'b) We believe that increasing government-funded public transportation subsidies will significantly improve the standard of living in the cantons.

Explanation: Text A suggests reducing social assistance benefits to encourage self-sufficiency, implying a preference for less government intervention in individuals' finances. Text B, on the other hand, advocates for increasing public transportation sub

Separate fact. inconsistency from explanation

In [24]:
def process_fact_inconsistency(input):
    res = input.split("b)")[1].split("Explanation:")
    text = res[0].strip()
    explanation = res[1].strip()
    return text, explanation

In [25]:
df['explanation_fact_inconsistency'] = 0

for i in range(len(df)):
    text, explanation = process_fact_inconsistency(df['factual_inconsistency'][i])
    df['explanation_fact_inconsistency'][i] = explanation
    df['factual_inconsistency'][i] = text

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_139208\3093813389.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['explanation_fact_inconsistency'][i] = explanation
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_139208\3093813389.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Text A suggests reducing social assistance benefits to encourage self-sufficiency, implying a preference for less government intervention in individuals' finances. Text B, on the other hand, advocates for increasing public transportation subsidies, which requires more government spending and contradicts the idea of minimizing state assistance. This requires an understanding of economic policy and the typical contradiction between reducing social b

In [27]:
df.head()

,text,topic,direct_support,direct_against,indirect_support,indirect_against,factual_inconsistency,explanation_fact_inconsistency
0,Support efforts to reduce social assistance be...,welfare,We advocate for reducing social assistance ben...,NaN,NaN,NaN,We believe that increasing government-funded p...,Text A suggests reducing social assistance ben...
1,Adjustment of social assistance guidelines wit...,welfare,We support revising social assistance guidelin...,NaN,NaN,NaN,We believe in increasing financial support for...,Text A indicates a policy that seeks to reduce...
2,Raising the retirement age for women and men t...,welfare,We support the proposal to increase the retire...,NaN,NaN,NaN,We believe in reducing the standard working we...,Text A proposes to increase the retirement age...
3,No disability pensions for pain disorders that...,welfare,We support the policy of not granting disabili...,NaN,NaN,NaN,We advocate for comprehensive health care cove...,Text A mentions a policy of not granting disab...
4,Introduction of four weeks of paid paternity l...,welfare,We advocate for the implementation of four wee...,NaN,NaN,NaN,We believe that new fathers should return to w...,Text A supports four weeks of paid paternity l...


In [28]:
df.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\Zurich_NLP\all_statements_de.csv")